In [0]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding,Bidirectional,LSTM,GlobalMaxPool1D,Dropout
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import save_model
import csv 
import pandas as pd 
from IPython.display import display, HTML
import h5py
from random import shuffle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd '/content/drive/My Drive/clickbait17-train-170331'

/content/drive/.shortcut-targets-by-id/1YESJbDd6eN2iexOWB-D2VlRKRYRQQa7T/clickbait17-train-170331


In [0]:
truth_data = pd.read_json("truth.jsonl", lines=True)
truth_data.head()
#testing data
Ttruth_data = pd.read_json("Ttruth.jsonl", lines=True)

In [0]:
clickbait_data = pd.read_json("instances.jsonl", lines=True)
clickbait_data.head()
#test data
Tclickbait_data = pd.read_json("Tinstances.jsonl", lines=True)

In [7]:
truth_data.head()

,truthJudgments,truthMean,id,truthClass,truthMedian,truthMode
0,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,858464162594172928,clickbait,1.000000,1.000000
1,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,858462320779026432,no-clickbait,0.000000,0.000000
2,"[0.33333333330000003, 0.6666666666000001, 1.0,...",0.400000,858460992073863168,no-clickbait,0.333333,0.000000
3,"[0.0, 0.6666666666000001, 0.0, 0.3333333333000...",0.266667,858459539296980992,no-clickbait,0.333333,0.333333
4,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,858455355948384256,no-clickbait,0.000000,0.000000


In [8]:
clickbait_data.head()

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has..."
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min..."


In [0]:
data = pd.merge(clickbait_data,truth_data, on = 'id')
data.head()
Tdata = pd.merge(Tclickbait_data,Ttruth_data, on = 'id')

In [10]:
data

,postMedia,postText,id,targetCaptions,targetParagraphs,targetTitle,postTimestamp,targetKeywords,targetDescription,truthJudgments,truthMean,truthClass,truthMedian,truthMode
0,[],[UK’s response to modern slavery leaving victi...,858462320779026432,[modern-slavery-rex.jpg],[Thousands of modern slavery victims have not ...,‘Inexcusable’ failures in UK’s response to mod...,Sat Apr 29 23:25:41 +0000 2017,"modern slavery, Department For Work And Pensio...",“Inexcusable” failures in the UK’s system for ...,"[0.33333333330000003, 0.0, 0.33333333330000003...",0.133333,no-clickbait,0.000000,0.000000
1,[],[this is good],858421020331560960,"[In this July 1, 2010 file photo, Dr. Charmain...",[President Donald Trump has appointed the pro-...,Donald Trump Appoints Pro-Life Advocate as Ass...,Sat Apr 29 20:41:34 +0000 2017,"Americans United for Life, Dr. Charmaine Yoest...",President Donald Trump has appointed pro-life ...,"[1.0, 1.0, 1.0, 1.0, 1.0]",1.000000,clickbait,1.000000,1.000000
2,[],"[The ""forgotten"" Trump roast: Relive his bruta...",858368123753435136,[President Trump will not attend this year's W...,[When the White House correspondents’ dinner i...,The ‘forgotten’ Trump roast: Relive his brutal...,Sat Apr 29 17:11:23 +0000 2017,"trump whcd, whcd, white house correspondents d...",President Trump won't be at this year's White ...,"[0.33333333330000003, 1.0, 0.33333333330000003...",0.466667,no-clickbait,0.333333,0.333333
3,[],[Meet the happiest #dog in the world!],858323428260139008,"[Maru , Maru, Maru, Maru, Maru]",[Adorable is probably an understatement. This ...,"Meet The Happiest Dog In The World, Maru The H...",Sat Apr 29 14:13:46 +0000 2017,"Maru, husky, dogs, pandas, furball, instagram","The article is about Maru, a husky dog who has...","[1.0, 0.6666666666000001, 1.0, 1.0, 1.0]",0.933333,clickbait,1.000000,1.000000
4,[],[Tokyo's subway is shut down amid fears over a...,858283602626347008,[All nine lines of Tokyo's subway system were ...,[One of Tokyo's major subways systems says it ...,Tokyo's subway is shut down amid fears over an...,Sat Apr 29 11:35:31 +0000 2017,"Tokyo,subway,shut,fears,North,Korean,attack","The temporary suspension, which lasted ten min...","[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,no-clickbait,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19533,[media/photo_804240867972304896.jpg],[Brazil soccer team and pilot's final intervie...,804250183642976256,"[CNBC, msnbc, NBC NEWS, TODAY, xfinity]",[Watch Live: Joe Biden Honored on Senate Floor...,"NBC News Video See Brazil Soccer Team, Pilot’s...",Thu Dec 01 09:06:00 +0000 2016,,NBC News,"[0.33333333330000003, 0.0, 1.0, 0.666666666600...",0.400000,no-clickbait,0.333333,0.000000
19534,[],[😱😱😱😱😱😱😱😱😱😱😱😱😱😱],804156272086020096,"[Instagram/madonna, Speaker Ryan Retreats on H...",[On November 30 Politico reported that Eric Tr...,Politico Scoop: Eric Trump Killed Two Deer,Thu Dec 01 02:52:50 +0000 2016,Politico Scoop: Eric Trump Killed Two Deer,Politico Scoop: Eric Trump Killed Two Deer,"[1.0, 1.0, 1.0, 1.0, 0.0]",0.800000,clickbait,1.000000,1.000000
19535,[],[Frenchs Forest high school may have to make w...,804149798651588608,[An artist's impression of the proposed new to...,[The Forest High School on Sydney's northern b...,Frenchs Forest high school may relocate to mak...,Thu Dec 01 02:27:07 +0000 2016,"frenchs forest, northern beaches, sydney, rede...",The Forest High School on Sydney's northern be...,"[0.0, 0.0, 0.0, 0.0, 0.0]",0.000000,no-clickbait,0.000000,0.000000
19536,[media/photo_804133521023324160.jpg],[Oh Jeff… #bruh],804134698729385984,[Jeff Fisher May Think Danny Woodhead Still Pl...,[NFL coaches have a lot of information to reme...,Los Angeles Rams Jeff Fisher May Think Danny W...,Thu Dec 01 01:27:06 +0000 2016,"Humor, Football, NFL, NFC West, Los Angeles Ra...","Los Angeles Rams news, rumors, scores, schedul...","[0.0, 0.0, 0.0, 0.0, 0.6666666666000001]",0.133333,no-clickbait,0.000000,0.000000


In [11]:
data['postText']

0        [UK’s response to modern slavery leaving victi...
1                                           [this is good]
2        [The "forgotten" Trump roast: Relive his bruta...
3                   [Meet the happiest #dog in the world!]
4        [Tokyo's subway is shut down amid fears over a...
                               ...                        
19533    [Brazil soccer team and pilot's final intervie...
19534                                     [😱😱😱😱😱😱😱😱😱😱😱😱😱😱]
19535    [Frenchs Forest high school may have to make w...
19536                                     [Oh Jeff… #bruh]
19537    [Richard Sherman weighs in on Cam Newton’s str...
Name: postText, Length: 19538, dtype: object

In [12]:
data['postText'][0]

['UK’s response to modern slavery leaving victims destitute while abusers go free']

In [13]:
data['targetDescription'][0]

'“Inexcusable” failures in the UK’s system for dealing with modern slavery are\xa0leaving victims reduced to destitution while their abusers go free because they are not adequately supported to testify against them, an alarming report has warned.'

In [14]:
data.shape

(19538, 14)

In [0]:
df = pd.DataFrame({'label':data.truthClass, 'text':data.postText})

In [0]:
Tdf = pd.DataFrame({'label':Tdata.truthClass, 'text':Tdata.postText})

In [17]:
df.head()

,label,text
0,no-clickbait,[UK’s response to modern slavery leaving victi...
1,clickbait,[this is good]
2,no-clickbait,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,clickbait,[Meet the happiest #dog in the world!]
4,no-clickbait,[Tokyo's subway is shut down amid fears over a...


In [0]:
# clean up 
# 2nd argument of replace must be str
def clean_up(label):
    label = label.replace("no-clickbait", "0")
    label = label.replace("clickbait", "1")
    return label

In [0]:
# convert to series and clean up the labels column
df["label"] = df.iloc[:,0].apply(clean_up)

In [0]:
l = []
for i in range(len(df)):
  l.append(int(df.iloc[:,0][i]))


In [0]:
df["label"] = l

In [22]:
df.head()

,label,text
0,0,[UK’s response to modern slavery leaving victi...
1,1,[this is good]
2,0,"[The ""forgotten"" Trump roast: Relive his bruta..."
3,1,[Meet the happiest #dog in the world!]
4,0,[Tokyo's subway is shut down amid fears over a...


In [0]:
t = []
for i in range(len(df)):
  t.append(df.iloc[:,1][i][0])


In [0]:
df["text"] = t

In [0]:
df['label'].value_counts()

0    14777
1     4761
Name: label, dtype: int64

In [0]:
Tdf['label'].value_counts()

no-clickbait    1697
clickbait        762
Name: label, dtype: int64

both datasets 1 and 2 are imbalanced since the number of non-clickbait tweets in datasets 1 and 2 are 2.1 and 3.1 times bigger than the number of clickbait tweets respectively.

In [0]:
#change df_trn and df_val
Tdf["label"] = Tdf.iloc[:,0].apply(clean_up)
l = []
for i in range(len(Tdf)):
  l.append(int(Tdf.iloc[:,0][i]))
Tdf["label"] = l

t = []
for i in range(len(Tdf)):
  t.append(Tdf.iloc[:,1][i][0])
Tdf["text"] = t

In [0]:
Tdf.head()

,label,text
0,0,Apple's iOS 9 'App thinning' feature will give...
1,0,RT @kenbrown12: Emerging market investors are ...
2,1,U.S. Soccer should start answering tough quest...
3,0,How theme parks like Disney World left the mid...
4,1,Could light bulbs hurt your health? One compan...


In [0]:
df_trn, df_val = Tdf, df

In [0]:
df_trn.shape, df_val.shape

((2459, 2), (19538, 2))

In [0]:
df_trn['text'][1]

'RT @kenbrown12: Emerging market investors are doing their best Monty Pythons--"Run away, run away"'

In [0]:
data_text = df_trn['text']

In [0]:
def tokenize(thresh = 5):
    count  = dict()
    idx = 1
    word_index = dict()
    for txt in data_text:
        words = process(txt)
        for word in words:
            if word in count.keys():
                count[word] += 1
            else:
                count[word]  = 1
    most_counts = [word for word in count.keys() if count[word]>=thresh]
    for word in most_counts:
        word_index[word] = idx
        idx+=1
    return word_index

In [0]:
import re
def process(txt):
    out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    out = out.split()
    out = [word.lower() for word in out]
    return out

In [0]:
num_words = None
word_index = tokenize()
num_words = len(word_index)
print('length of the dictionary ',len(word_index))

length of the dictionary  1111


In [0]:
def getMax(data):
    max_tokens = 0 
    for txt in data:
        if max_tokens < len(txt.split()):
            max_tokens = len(txt.split())
    return max_tokens
max_tokens = getMax(df_trn['text'])

In [0]:
def create_sequences(data):
    tokens = []
    for txt in data:
        words = process(txt)
        seq = [0] * max_tokens
        i = 0 
        for word in words:
            start = max_tokens-len(words)
            if word.lower() in word_index.keys():
                seq[i+start] = word_index[word]
            i+=1
        tokens.append(seq)        
    return np.array(tokens)

In [0]:
x_train_tokens = create_sequences(df_trn['text'])

In [0]:
x_test_tokens = create_sequences(df_val['text'])

In [0]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model.add(GRU(units=16, name = "gru_1",return_sequences=True))
model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
model.add(GRU(units=4, name= "gru_3"))
model.add(Dense(1, activation='sigmoid',name="dense_1"))
optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
model.fit(x_train_tokens, df_trn['label'], validation_split=0.05, epochs=3, batch_size=32)

Train on 2336 samples, validate on 123 samples
Epoch 1/3
2336/2336 [==============================] - 8s 4ms/sample - loss: 0.6397 - acc: 0.6858 - val_loss: 0.6135 - val_acc: 0.6911
Epoch 2/3
2336/2336 [==============================] - 7s 3ms/sample - loss: 0.6012 - acc: 0.6901 - val_loss: 0.5913 - val_acc: 0.6911
Epoch 3/3
2336/2336 [==============================] - 7s 3ms/sample - loss: 0.5130 - acc: 0.7466 - val_loss: 0.5762 - val_acc: 0.6992


In [0]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(x_test_tokens, df_val['label'], batch_size=128)
print('test loss, test acc:', results)


# Evaluate on test data
19538/19538 [==============================] - 3s 167us/sample - loss: 0.4856 - acc: 0.7624
test loss, test acc: [0.48564924422202693, 0.7623605]


In [0]:
txt = ["Hello  Clickbait is the yashwant id s lewkd","Terrible movie","would","I like that movie"]
print(create_sequences(txt)[0])
pred = model.predict(create_sequences(txt))
print('\n prediction for \n',pred[:,0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 38 31  0  0  0  0]

 prediction for 
 [0.554967 0.408303 0.455486 0.454243]


In [0]:
save_model(
    model,
    "keras.h5",
    overwrite=True,
    include_optimizer=True
)

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 27, 8)             8888      
_________________________________________________________________
gru_1 (GRU)                  (None, 27, 16)            1200      
_________________________________________________________________
gru_2 (GRU)                  (None, 27, 8)             600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 10,849
Trainable params: 10,849
Non-trainable params: 0
_________________________________________________________________


In [0]:
!pip install tensorflowjs

In [0]:
!tensorflowjs_converter --input_format keras k.h5 my_model_as_tfjs1